# This notebook is for recalculating the indexes which are corresponding to positions in the sequences after removal of unreferenced indexes.

In [1]:
import pandas as pd
import ast
import re

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
df.head()

,Unnamed: 0,acc,mobidb_consensus,shortened_mobidb_consensus,ncbi_taxon_id,organism,Sequence Length,sequence,#_AA_referenced,Fraction Missing,shortened_sequence,Updated Indexes,D_Class,S_Class,C_Class,Total_Disordered_Regions,Total_Ordered_Regions,Total_Ambiguous_Regions
0,0,P31994,"{'disorder': {'derived': [{'regions': [[46, 46...","[[45, 45, 'D'], [46, 49, 'C'], [50, 66, 'S'], ...",9606,Homo sapiens (Human),310,MGILSFLPVLATESDWADCKSPQPWGHMLLWTAVLFLAPVAGTPAA...,173,0.441935,AAPPKAVLKLEPQWINVLQEDSVTLTCRGTHSPESDSIQWFHNGNL...,"[[1, 1, 'D'], [2, 5, 'C'], [6, 22, 'S'], [23, ...","[45, 45, 'D']","[50, 66, 'S'],[102, 103, 'S'],[107, 108, 'S'],...","[46, 49, 'C'],[67, 101, 'C'],[104, 106, 'C'],[...",1,88,84
1,1,F2Z602,"{'disorder': {'derived': [{'regions': [[2, 143...","[[1, 1, 'D'], [2, 143, 'S'], [144, 145, 'C']]",284590,Kluyveromyces lactis (strain ATCC 8585 / CBS 2...,145,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,145,0.000000,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,"[[1, 1, 'D'], [2, 143, 'S'], [144, 145, 'C']]","[1, 1, 'D']","[2, 143, 'S']","[144, 145, 'C']",1,142,2
2,2,Q875M3,"{'disorder': {'derived': [{'regions': [[1, 145...","[[1, 145, 'S']]",28985,Kluyveromyces lactis (Yeast) (Candida sphaerica),145,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,145,0.000000,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,"[[1, 145, 'S']]",NaN,"[1, 145, 'S']",NaN,0,145,0
3,3,Q86A17,"{'disorder': {'derived': [{'regions': [[2, 2, ...","[[1, 1, 'D'], [2, 2, 'C'], [3, 37, 'S'], [38, ...",44689,Dictyostelium discoideum (Slime mold),231,MSKNILVLGGSGALGAEVVKFFKSKSWNTISIDFRENPNADHSFTI...,231,0.000000,MSKNILVLGGSGALGAEVVKFFKSKSWNTISIDFRENPNADHSFTI...,"[[1, 1, 'D'], [2, 2, 'C'], [3, 37, 'S'], [38, ...","[1, 1, 'D']","[3, 37, 'S'],[40, 40, 'S'],[42, 62, 'S'],[66, ...","[2, 2, 'C'],[38, 39, 'C'],[41, 41, 'C'],[63, 6...",1,223,7
4,4,P10855,"{'disorder': {'derived': [{'regions': [[30, 91...","[[24, 29, 'D'], [30, 91, 'S'], [92, 92, 'D']]",10090,Mus musculus (Mouse),92,MKVSTTALAVLLCTMTLCNQVFSAPYGADTPTACCFSYSRKIPRQF...,69,0.250000,APYGADTPTACCFSYSRKIPRQFIVDYFETSSLCSQPGVIFLTKRN...,"[[1, 6, 'D'], [7, 68, 'S'], [69, 69, 'D']]","[24, 29, 'D'],[92, 92, 'D']","[30, 91, 'S']",NaN,7,62,0


In [4]:
shortenedSeq_list =[] # defining new lists
updatedIndexes = [] 

for rows in df.itertuples(): # iterate through rows
    sequence = rows.sequence # assign protein sequence to variable
    classReferences = rows.shortened_mobidb_consensus # assign numbers and corresponding class to variable
    result = ast.literal_eval(classReferences)
    newSequence = ''
    
    for item in result: # iterate through class references
        newSequence += sequence[item[0]-1:item[1]] # remove amino acids which are not referenced
    shortenedSeq_list.append(newSequence)

    # Update the class reference based on removed amino acids
    currentIndex = 1
    for i in range(len(result)): # for each of the class references 
        reference = result[i]
        if reference[0] != currentIndex: # if the first reference number does not correspond to current index
            difference = reference[0] - currentIndex # calculate the difference between reference number and current index
            reference[0] -= difference # update the indexes in the reference
            reference[1] -= difference
        currentIndex = reference[1] + 1 # update current index to move onto next set of references
        updatedIndexes.append(reference) # add updated indexes to list

In [5]:
# Creating a new Dataframe
df2 = pd.DataFrame()

In [6]:
# Checking that updated indexes were formed correctly
df2['test'] = updatedIndexes

In [7]:
# Extracting the letters from the references
df['lettersOnlyTest'] = df['shortened_mobidb_consensus'].str.replace(r'[[\d\,\s\]]', '')

/Users/rebeccalenihan/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:619: FutureWarning: Possible nested set at position 1
  compiled = re.compile(pat, flags=flags)


In [8]:
df['lettersOnlyTest']

0        'D''C''S''C''S''C''S''C''S''C''S''C''S''C''S''...
1                                                'D''S''C'
2                                                      'S'
3                              'D''C''S''C''S''C''S''C''S'
4                                                'D''S''D'
                               ...                        
40082                                                  'S'
40083                                      'C''S''C''S''C'
40084                                         'D''C''S''C'
40085                                                  'S'
40086                                                  'S'
Name: lettersOnlyTest, Length: 40087, dtype: object

In [9]:
letterList = []

# Adding letters to a list 
for rows in df.itertuples():
    myletterList = rows.lettersOnlyTest
    toString = ''.join(myletterList)
    onlyLetters = re.findall(r'[a-zA-z]', toString)

    letterList.append(onlyLetters)

letterList

[['D',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C'],
 ['D', 'S', 'C'],
 ['S'],
 ['D', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S'],
 ['D', 'S', 'D'],
 ['C', 'S', 'C', 'S'],
 ['S', 'C', 'S', 'C', 'S', 'C', 'S', 'C'],
 ['S'],
 ['S'],
 ['D', 'S', 'C'],
 ['C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S'],
 ['C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'D'],
 ['D', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S'],
 ['S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'C',
  'D',
  'S',
  'C',
  'D',
  'C',
  'S',
  'C',
  'S',
  'C'],
 ['S', 'C', 'S', 'C', 'D', 'S', 'C', 'S', 'C', 'D'],
 ['C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'D'],
 ['S', 'C', 'S', 'C', 'D'],
 ['S'],
 ['S', 'C', 'S', 'C'],
 ['S', 'D', 'S', 'D'],
 ['S', 'D', 'S', 'C', 'S'],
 ['D', 'C', 'S', 'C', 'S', 'C', 'S', 'C'],
 ['D', 'S', 'C', 'S', 'D'],
 ['D', 'C', 'S', 'C', 'S', 'C', '

In [10]:
# Formatting list 
flat_list2 = []
for sublist in letterList:
    for item in sublist:
        flat_list2.append(item)

In [18]:
newIndex, c = [], 0
for i in [len(x) for x in letterList]:
    newIndex.append(updatedIndexes[c: c + i])
    c += i

In [19]:
# Adding new intervals as column in dataframe
df['Test intervals'] = newIndex

In [20]:
df.head(30)

,Unnamed: 0,acc,mobidb_consensus,shortened_mobidb_consensus,ncbi_taxon_id,organism,Sequence Length,sequence,#_AA_referenced,Fraction Missing,shortened_sequence,Updated Indexes,D_Class,S_Class,C_Class,Total_Disordered_Regions,Total_Ordered_Regions,Total_Ambiguous_Regions,lettersOnlyTest,Test intervals
0,0,P31994,"{'disorder': {'derived': [{'regions': [[46, 46...","[[45, 45, 'D'], [46, 49, 'C'], [50, 66, 'S'], ...",9606,Homo sapiens (Human),310,MGILSFLPVLATESDWADCKSPQPWGHMLLWTAVLFLAPVAGTPAA...,173,0.441935,AAPPKAVLKLEPQWINVLQEDSVTLTCRGTHSPESDSIQWFHNGNL...,"[[1, 1, 'D'], [2, 5, 'C'], [6, 22, 'S'], [23, ...","[45, 45, 'D']","[50, 66, 'S'],[102, 103, 'S'],[107, 108, 'S'],...","[46, 49, 'C'],[67, 101, 'C'],[104, 106, 'C'],[...",1,88,84,'D''C''S''C''S''C''S''C''S''C''S''C''S''C''S''...,"[[1, 1, D], [2, 5, C], [6, 22, S], [23, 57, C]..."
1,1,F2Z602,"{'disorder': {'derived': [{'regions': [[2, 143...","[[1, 1, 'D'], [2, 143, 'S'], [144, 145, 'C']]",284590,Kluyveromyces lactis (strain ATCC 8585 / CBS 2...,145,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,145,0.000000,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,"[[1, 1, 'D'], [2, 143, 'S'], [144, 145, 'C']]","[1, 1, 'D']","[2, 143, 'S']","[144, 145, 'C']",1,142,2,'D''S''C',"[[1, 1, D], [2, 143, S], [144, 145, C]]"
2,2,Q875M3,"{'disorder': {'derived': [{'regions': [[1, 145...","[[1, 145, 'S']]",28985,Kluyveromyces lactis (Yeast) (Candida sphaerica),145,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,145,0.000000,MGKGKPRGLNSARKLRVHRRNNRWAETTYKKRLLGTAFKSSPFGGS...,"[[1, 145, 'S']]",NaN,"[1, 145, 'S']",NaN,0,145,0,'S',"[[1, 145, S]]"
3,3,Q86A17,"{'disorder': {'derived': [{'regions': [[2, 2, ...","[[1, 1, 'D'], [2, 2, 'C'], [3, 37, 'S'], [38, ...",44689,Dictyostelium discoideum (Slime mold),231,MSKNILVLGGSGALGAEVVKFFKSKSWNTISIDFRENPNADHSFTI...,231,0.000000,MSKNILVLGGSGALGAEVVKFFKSKSWNTISIDFRENPNADHSFTI...,"[[1, 1, 'D'], [2, 2, 'C'], [3, 37, 'S'], [38, ...","[1, 1, 'D']","[3, 37, 'S'],[40, 40, 'S'],[42, 62, 'S'],[66, ...","[2, 2, 'C'],[38, 39, 'C'],[41, 41, 'C'],[63, 6...",1,223,7,'D''C''S''C''S''C''S''C''S',"[[1, 1, D], [2, 2, C], [3, 37, S], [38, 39, C]..."
4,4,P10855,"{'disorder': {'derived': [{'regions': [[30, 91...","[[24, 29, 'D'], [30, 91, 'S'], [92, 92, 'D']]",10090,Mus musculus (Mouse),92,MKVSTTALAVLLCTMTLCNQVFSAPYGADTPTACCFSYSRKIPRQF...,69,0.250000,APYGADTPTACCFSYSRKIPRQFIVDYFETSSLCSQPGVIFLTKRN...,"[[1, 6, 'D'], [7, 68, 'S'], [69, 69, 'D']]","[24, 29, 'D'],[92, 92, 'D']","[30, 91, 'S']",NaN,7,62,0,'D''S''D',"[[1, 6, D], [7, 68, S], [69, 69, D]]"
5,5,P06608,"{'disorder': {'derived': [{'regions': [[22, 22...","[[22, 24, 'C'], [25, 38, 'S'], [39, 55, 'C'], ...",556,Dickeya chrysanthemi (Pectobacterium chrysanth...,348,MERWFKSLFVLVLFFVFTASAADKLPNIVILATGGTIAGSAATGTQ...,327,0.060345,ADKLPNIVILATGGTIAGSAATGTQTTGYKAGALGVDTLINAVPEV...,"[[1, 3, 'C'], [4, 17, 'S'], [18, 34, 'C'], [35...",NaN,"[25, 38, 'S'],[56, 348, 'S']","[22, 24, 'C'],[39, 55, 'C']",0,307,20,'C''S''C''S',"[[1, 3, C], [4, 17, S], [18, 34, C], [35, 327,..."
6,6,P12995,"{'disorder': {'derived': [{'regions': [[1, 429...","[[1, 159, 'S'], [160, 168, 'C'], [169, 182, 'S...",83333,Escherichia coli (strain K12),429,MTTDDLAFDQRHIWHPYTSMTSPLPVYPVVSAEGCELILSDGRRLV...,429,0.000000,MTTDDLAFDQRHIWHPYTSMTSPLPVYPVVSAEGCELILSDGRRLV...,"[[1, 159, 'S'], [160, 168, 'C'], [169, 182, 'S...",NaN,"[1, 159, 'S'],[169, 182, 'S'],[184, 299, 'S'],...","[160, 168, 'C'],[183, 183, 'C'],[300, 301, 'C'...",0,416,13,'S''C''S''C''S''C''S''C',"[[1, 159, S], [160, 168, C], [169, 182, S], [1..."
7,7,P45697,"{'disorder': {'derived': [{'regions': [[20, 83...","[[20, 83, 'S']]",34649,Mesobuthus martensii (Manchurian scorpion) (Bu...,84,MNYLVMISFALLLMTGVESVRDAYIAKPHNCVYECARNEYCNDLCT...,64,0.238095,VRDAYIAKPHNCVYECARNEYCNDLCTKNGAKSGYCQWVGKYGNGC...,"[[1, 64, 'S']]",NaN,"[20, 83, 'S']",NaN,0,64,0,'S',"[[1, 64, S]]"
8,8,O67023,"{'disorder': {'derived': [{'regions': [[2, 71,...","[[1, 216, 'S']]",224324,Aquifex aeolicus (strain VF5),216,MLKIGVLVSGRGSNLQAIIDAIES